# Sequence generation

In [1]:
import pandas as pd
import pyreadr
import numpy as np
import torch
import pickle
from os import listdir
from os.path import isfile, join

In [2]:
patient = pyreadr.read_r('../ServerData_13Oct2020/d_patient_overall.Rdata')
practice = pyreadr.read_r('../ServerData_13Oct2020/d_practice.Rdata')
patient = patient['d_patient_overall']
practice = practice['d_practice']

#Age in 2016-01-01
patient['age'] = patient.year_of_birth.apply(lambda x: 2016-x)

patient = patient[['patid', 'practice_id', 'age']].merge(practice[['practice_id', 'Country']], how='left', on='practice_id')

# Outcomes data
outcomes = pd.read_csv("../FinalData/cleaned_outcomes_11072023.csv")

/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[col

In [3]:
outcomes['new_12MonthsOutcome'] = outcomes.apply(lambda x: (x.outcome_15months)|(x.outcome_18months)|(x.outcome_21months)|(x.outcome_24months), axis=1)

In [ ]:
#load clinical information
path = '../ServerData_13Oct2020/'
clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
clinical = pyreadr.read_r('../ServerData_13Oct2020/f_clinical_part1.Rdata')
clinical = clinical['f_clinical_part']
for file in clinical_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_clinical_part']
    print(temp.shape)
    clinical = pd.concat([clinical, temp])
    clinical.reset_index(drop=True, inplace=True)
    
#data selection 
clinical = clinical.dropna(subset=['code_id'])
clinical['event_date'] = pd.to_datetime(clinical['event_date'])
clinical = clinical.loc[(clinical['event_date'] >= '2016-01-01') & (clinical['event_date'] < '2018-01-01')]

#add paractice information

#load clinical information
path = '../ServerData_13Oct2020/'
therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
therapy = pyreadr.read_r('../ServerData_13Oct2020/f_therapy_part1.Rdata')
therapy = therapy['f_therapy_part']
for file in therapy_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_therapy_part']
    print(temp.shape)
    therapy = pd.concat([therapy, temp])
    therapy.reset_index(drop=True, inplace=True)
    
#data selection 
therapy = therapy.dropna(subset=['code_id'])
therapy['event_date'] = pd.to_datetime(therapy['event_date'])
therapy = therapy.loc[(therapy['event_date'] >= '2016-01-01') & (therapy['event_date'] < '2018-01-01')]

In [ ]:
all_raw_data = pd.concat([clinical[['patid', 'event_date', 'code_id']],
           therapy[['patid', 'event_date', 'code_id']]])
all_raw_data.reset_index(drop=True, inplace=True)
#vocab and code2idx generation
vocab_all = all_raw_data.code_id.unique().tolist()
idx_all = range(1, len(vocab_all)+1)

code2idx_all = dict(zip(vocab_all, idx_all))
idx2code_all = dict(zip(idx_all, vocab_all))

code2idx_all['PAD'] = 0
idx2code_all[0] = 'PAD'
VOCAB_SIZE= len(code2idx_all)
print('code2idx Size: {}'.format(len(code2idx_all)))
print('idx2code Size: {}'.format(len(idx2code_all)))
# print(code2idx)
#extract year, month, day from event date
all_raw_data['day'] = all_raw_data.apply(lambda x: str(x['event_date'].day), axis=1)
all_raw_data['month'] = all_raw_data.apply(lambda x: str(x['event_date'].month), axis=1)
all_raw_data['year'] = all_raw_data.apply(lambda x: str(x['event_date'].year), axis=1)
all_raw_data['read_code_seq'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
event_data_seq_all = all_raw_data.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()
all_raw_data = all_raw_data.drop_duplicates(subset=['patid']).reset_index(drop=True)     
data_all = all_raw_data[['patid', 'read_code_seq']].merge(event_data_seq_all, how='left',  on='patid')
data_all = data_all.merge(outcomes[['patid', 'new_12MonthsOutcome',
                           'outcome_combined_24months']], how='inner', on='patid')

data_all['read_code_seq'] = data_all['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data_all['length_read_code_seq'] = data_all['read_code_seq'].apply(lambda x: len(x))
data_all = data_all[data_all.length_read_code_seq > 10]

#padding at the beginning of the list
max_seq = 150
def make_uniform_data(x):
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x
    
data_all['read_code_seq_padded'] = data_all['read_code_seq'].apply(lambda x: make_uniform_data(x))
data_all['read_code_seq_padded_idx'] = data_all['read_code_seq_padded'].apply(lambda x: [code2idx_all.get(key) for key in x])
data_all = data_all.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data_all = data_all.drop_duplicates(subset=['patid']).reset_index(drop=True)     

#SAVE all important materials
pickle.dump(code2idx_all, open('../SeqModel/code2idx_all_new.sav', 'wb'))
pickle.dump(idx2code_all, open('../SeqModel/idx2code_all_new.sav', 'wb'))
pickle.dump(data_all, open('../SeqModel/data_all_new.sav', 'wb'))
# data_all.to_csv('../SeqModel/seqData_all.csv', index_label=False, index=False)

# LSTM Model using Clinical + Therapy sequence

In [20]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, SensitivityAtSpecificity
from tensorflow.keras.optimizers import Adam, Adagrad, RMSprop
from tensorflow.keras.initializers import Constant
from tensorflow.keras.regularizers import L1L2, L1, L2


#internal validation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, f1_score, balanced_accuracy_score, matthews_corrcoef, auc, average_precision_score, roc_auc_score, balanced_accuracy_score, roc_curve, accuracy_score

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pickle

# fix random seed for reproducibility
tf.random.set_seed(1234)

target_outcome = 'new_12MonthsOutcome'
max_codes = 150

In [7]:
# load the dataset but only keep the top n words, zero the rest
data = pickle.load(open('../SeqModel/data_all_new.sav', 'rb'))
code2idx = pickle.load(open('../SeqModel/code2idx_all_new.sav', 'rb'))
idx2code = pickle.load(open('../SeqModel/idx2code_all_new.sav', 'rb'))

vocab_size = len(code2idx)
vocab_size

37644

In [ ]:
# #Data split conventional (mixed countries)
# trainingData, testData = train_test_split(data, test_size=0.1, stratify=data[target_outcome], random_state=1234)
# trainingData, valData = train_test_split(trainingData, test_size=0.2, stratify=trainingData[target_outcome], random_state=1234)
# print(trainingData.shape)
# print(valData.shape)
# print(testData.shape)

In [9]:
#Data split, train=England, eval=Scot+Wales
trainingData = data[(data.Country == 'England') & (data.age >= 18)]
trainingData, valData = train_test_split(trainingData, test_size=0.2, stratify=trainingData[target_outcome], random_state=1234)
trainingData, evalData = train_test_split(trainingData, test_size=0.2, stratify=trainingData[target_outcome], random_state=1234)
testData = data[((data.Country == 'Wales') | (data.Country == 'Scotland')) & (data.age >= 18)]
testDataWales = data[(data.Country == 'Wales') & (data.age >= 18)]
testDataScotland = data[(data.Country == 'Scotland') & (data.age >= 18)]

In [10]:
print('Train: ', trainingData.shape[0])
print('Val: ', valData.shape[0])
print('Eval (internal validation): ', evalData.shape[0])
print('Test: ', testData.shape[0])
print('Test - Wales: ', testDataWales.shape[0])
print('Test - Scotland: ', testDataScotland.shape[0])

Train:  14031
Val:  4385
Eval (internal validation):  3508
Test:  1590
Test - Wales:  1062
Test - Scotland:  528


In [11]:
print(trainingData.patid.unique().shape)
print(trainingData.shape)

(14031,)
(14031, 12)


In [12]:
#make sure no data leak between sets
print(list(set(trainingData.patid.values).intersection(set(valData.patid.values))))
print(list(set(trainingData.patid.values).intersection(set(evalData.patid.values))))
print(list(set(valData.patid.values).intersection(set(evalData.patid.values))))
print(list(set(valData.patid.values).intersection(set(testData.patid.values))))
print(list(set(trainingData.patid.values).intersection(set(testData.patid.values))))
print(list(set(testData.patid.values).intersection(set(testDataScotland.patid.values)))) # here data leak is expected

[]
[]
[]
[]
[]
[43487233, 43487235, 43464708, 43139079, 43378699, 43139096, 43165721, 43378721, 43165731, 43165732, 43487269, 43165737, 43165746, 43325496, 43311166, 43374655, 43464772, 43227211, 43466834, 43378773, 43311191, 43339865, 43139165, 43284586, 43233387, 43374709, 43139214, 43315349, 43165847, 43430042, 43442340, 43145381, 43327661, 43350191, 43491513, 43315386, 43135161, 43145404, 43253954, 43430083, 43192522, 43190477, 43339982, 43190478, 43253966, 43303126, 43423963, 43430108, 43426012, 43430118, 43180267, 43251950, 43317488, 43141371, 43340029, 43317502, 43190531, 43317508, 43190533, 43204870, 43491591, 43340044, 43317518, 43430159, 43190548, 43317526, 43315483, 43208987, 43315486, 43317538, 43254052, 43208998, 43192615, 43317543, 43209011, 43430199, 43430210, 43413830, 43489611, 43340108, 43209038, 43491667, 43340118, 43477336, 43491680, 43491683, 43272551, 43413865, 43221371, 43426174, 43342206, 43413898, 43336077, 43209102, 43489679, 43342222, 43489683, 43426195, 4348

In [13]:
print(trainingData[target_outcome].value_counts(normalize=True))
print(valData[target_outcome].value_counts(normalize=True))
print(evalData[target_outcome].value_counts(normalize=True))
print(testData[target_outcome].value_counts(normalize=True))
print(testDataWales[target_outcome].value_counts(normalize=True))
print(testDataScotland[target_outcome].value_counts(normalize=True))

0    0.847195
1    0.152805
Name: new_12MonthsOutcome, dtype: float64
0    0.847206
1    0.152794
Name: new_12MonthsOutcome, dtype: float64
0    0.847206
1    0.152794
Name: new_12MonthsOutcome, dtype: float64
0    0.783648
1    0.216352
Name: new_12MonthsOutcome, dtype: float64
0    0.746704
1    0.253296
Name: new_12MonthsOutcome, dtype: float64
0    0.857955
1    0.142045
Name: new_12MonthsOutcome, dtype: float64


In [14]:
# X and y
X_train = np.array(trainingData.read_code_seq_padded_idx.values)
X_train = np.array([x for x in X_train])
X_val = np.array(valData.read_code_seq_padded_idx.values)
X_val = np.array([x for x in X_val])
X_eval = np.array(evalData.read_code_seq_padded_idx.values)
X_eval = np.array([x for x in X_eval])
X_test = np.array(testData.read_code_seq_padded_idx.values)
X_test = np.array([x for x in X_test])
X_testWales = np.array(testDataWales.read_code_seq_padded_idx.values)
X_testWales = np.array([x for x in X_testWales])
X_testScotland = np.array(testDataScotland.read_code_seq_padded_idx.values)
X_testScotland = np.array([x for x in X_testScotland])

y_train = trainingData[target_outcome].values
y_val = valData[target_outcome].values
y_eval = evalData[target_outcome].values
y_test = testData[target_outcome].values
y_testWales = testDataWales[target_outcome].values
y_testScotland = testDataScotland[target_outcome].values

In [15]:
print('Train: ', X_train.shape[0])
print('Val: ', X_val.shape[0])
print('Eval (internal validation): ', X_eval.shape[0])
print('Test: ', X_test.shape[0])
print('Test - Wales: ', X_testWales.shape[0])
print('Test - Scotland: ', X_testScotland.shape[0])

Train:  14031
Val:  4385
Eval (internal validation):  3508
Test:  1590
Test - Wales:  1062
Test - Scotland:  528


In [16]:
pos_weight = trainingData[target_outcome].value_counts()[0]/trainingData[target_outcome].value_counts()[1]
neg_weight = trainingData[target_outcome].value_counts()[1]/trainingData[target_outcome].value_counts()[0]
class_weight = {0:1, 1:pos_weight}
print(class_weight)

{0: 1, 1: 5.544309701492537}


In [17]:
output_bias = np.array([np.log(neg_weight)])
output_bias = Constant(output_bias)

In [ ]:
%%time

# create the model
embedding_vector_length = 175
earlyStopping = EarlyStopping(monitor='val_auc', patience=3, verbose=0, mode='max', restore_best_weights=True)
mcp_save = ModelCheckpoint('../SeqModel/seqModel_all_new.mdl_wts.hdf5', save_best_only=True, monitor='val_auc', mode='min')


with tf.device('/CPU:0'):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_codes))
    model.add(LSTM(100, return_sequences=True, dropout=0.5, recurrent_dropout=0.7, 
                   recurrent_activation='relu', bias_regularizer=L1L2(l1=0.0, l2=0.01)))
    model.add(LSTM(50, dropout=0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid', bias_initializer=output_bias))
    opt = RMSprop(learning_rate=0.0001)
    metrics = [
        AUC(num_thresholds=10000, name='auc', curve='ROC'),
    ]
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=metrics, )
    print(model.summary())
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=100, class_weight = class_weight, callbacks = [earlyStopping, mcp_save])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 175)          6587700   
                                                                 
 lstm (LSTM)                 (None, 150, 100)          110400    
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dense (Dense)               (None, 64)                3264      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6731629 (25.68 MB)
Trainable params: 6731629 (25.68 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/1

2023-10-26 11:41:37.195875: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7d7ce929f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-10-26 11:41:37.195965: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-10-26 11:41:37.208548: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-26 11:41:37.230838: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-10-26 11:41:37.232664: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-10-26 11:41:37.237675: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


  2/141 [..............................] - ETA: 38s - loss: 2.6505 - auc: 0.4815 

2023-10-26 11:41:37.637761: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


  3/141 [..............................] - ETA: 37s - loss: 2.8364 - auc: 0.5244

2023-10-26 11:41:37.904929: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


 47/141 [=========>....................] - ETA: 24s - loss: 2.7194 - auc: 0.5142

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
# plt.ylim(0.55,1)
plt.title('model AUC')
plt.ylabel('AUC')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
# plt.ylim(0.1, 1.15)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
with tf.device('/CPU:0'):
    model.evaluate(X_eval, y_eval)
    model.evaluate(X_test, y_test)
    model.evaluate(X_testWales, y_testWales)
    model.evaluate(X_testScotland, y_testScotland)

In [ ]:
#Model evaluation function
def summariseResult (testY, preds):
    tn, fp, fn, tp = confusion_matrix(testY, preds).ravel()
    specificity = tn / (tn+fp)
    sensitivity = tp / (tp+fn)
    ppv = 100*tp/(tp+fp)
    npv = 100*tn/(fn+tn)
    acc = accuracy_score(testY, preds)
    f1score = f1_score(testY, preds, average = 'binary')
    balanceacc = balanced_accuracy_score(testY, preds)
    fpr, tpr, thresholds = roc_curve(testY, preds, pos_label=1)
    aucscore = auc(fpr, tpr)
    # aucscore = roc_auc_score(testY, preds)
    auprc = average_precision_score(testY, preds)
    # plot_confusion_matrix(model, testX, testY, cmap='viridis')  
    return np.round(acc,4), np.round(specificity,4), np.round(sensitivity,4), np.round(aucscore,4), np.round(auprc,4), np.round(balanceacc,4), np.round(f1score,4), np.round(ppv,4), np.round(npv,4)

data_test_Xs = [X_eval, X_test, X_testWales, X_testScotland]
data_test_ys = [y_eval, y_test, y_testWales, y_testScotland]
for data_test_X, data_test_y in zip(data_test_Xs, data_test_ys):
    with tf.device('/CPU:0'):
        preds = model.predict(data_test_X)
    preds = [0 if pred <0.5 else 1 for pred in preds]
    print(summariseResult(data_test_y, np.squeeze(preds)))

In [ ]:
model.save('../SeqModel/model_all.h5')